In [ ]:
import nltk
import pandas as pd
import re
import random
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import stopwords
#transform
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

df = pd.read_csv('df_comb_v5_with_mood.csv')
print(df.columns)
df.head()


Clustering by the song's sound features
- danceability
- energy
- key
- loudness
- mode
- speechiness
- acousticness
- instrumentalness
- liveness
- valence
- tempo
Run Elbow Method to determine what's the best cluster number for the dataset

In [ ]:
from sklearn.cluster import KMeans

clustering_cols = ['danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo']
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(df[clustering_cols])
    wcss.append(kmeans.inertia_)
plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

From the above plot, there would be 4 clusters.
Then run the KMeans to assign the cluster to each song and add the dataframe

In [ ]:
kmeans = KMeans(n_clusters=4)
predicted_cluster = kmeans.fit_predict(df[clustering_cols])
df['cluster'] = predicted_cluster
print(df['cluster'].value_counts())
df.head()
df.to_csv('df_comb_v5_clustered.csv', index=False)